# EC tower (processed) data extraction
A simple script used in python to extract information from the EC tower (processed) data, a part of the GRAPEX project. The product from this programm can be used to generate the corresponding footprint area, and can also (conveniently) used for TSEB modeling verification.
## Brief description
- Running this script successfully, requires an EXCEL table providing a searching index in order to find the correct record from the EC-tower processed table. Year, month, day, hour, and minute are keys to locating the record in the EC-tower after processed data (table). "Request_Demo.xlsx" is an example.<br>
- "Demo_Request.xlsx" is demo data from users' requests. "Year", "Month", "Day", and "Time_EC" are hard required. "Time_EC" represents the time for the EC-tower records, and the time can only be on the hour or half hour. Because the EC-tower only provides processed data in the middle between two neighbor o'clock, interpretation was done on the hour based on the neighbor's half hours. "Time_EC" was approximated based on the "Time_flight", which stood for the AggieAir (https://uwrl.usu.edu/aggieair/) flight time.
- Manual work is still required since the processed tables from the GRAPEX project are not exactly the same. Thus, slight modification for this script or the original table (not recommended) is required accordingly.

Create date: October 14th, 2021<br>
Latest update date: October 24th, 2021<br>
Main contact: Rui.Gao@usu.edu<br>

In [9]:
import pandas as pd
import numpy as np

In [10]:
# Manually modify the output table name and the output folder path
dir_output = r'D:\...\3_Results'
NameCSV = 'OUT_Extracted'
EC_monitoring_height = 4.0
Longitude = -120.1761 # flux-tower longitude (example)
Latitude = 36.8492 # flux-tower latitude (example)

In [11]:
# It needs manual work to make sure the table you read is correct
# This is because that the format of the EC-tower processed data in the GRAPEX project is not the same for each research site.
# "sheet_name" and "header" need to modify accordingly.
Table_EC_Obs = pd.read_excel(r'D:\...\1_Data\GRAPEX_EC_Tower_Demo.xlsx',
                             sheet_name='R720 #1',
                             header=4)
Table_EC_Obs.head(3)

,Unnamed: 0,Year,Month,Day,Hour,Minute,DOY,TOD,DEC,Wind\nSpeed,...,Net\nRadiation,Soil\nHeat Flux,Sensible\nHeat Flux,Latent\nHeat Flux,CO2\nFlux,ustar,Surface\nTemp,Soil\nTemp,Soil \nMoisture\nContent,Battery
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[m s-1],...,[W m-2],[W m-2],[W m-2],[W m-2],[mg m-2 s-1],[m s-1],[°C],[°C],[%],[V]
1,NaN,2018.0,5.0,19.0,0.0,30.0,139.0,0.5,139.02083,1.49,...,-61.9,-35.89,-29.85,6.65,0.1917,0.2022,16.1,20.58,18.46,12.48
2,NaN,2018.0,5.0,19.0,1.0,30.0,139.0,1.5,139.06250,1.14,...,-54.65,-38.62,-21.8,4.83,0.1804,0.1663,15.13,20.04,18.46,12.44


In [12]:
# Read the request table (Time is the key in this table)
Table_Request = pd.read_excel(r'D:\...\1_Data\Request_Demo.xlsx')
Table_Request.head(3)

,Site,Year,Month,Day,Time_flight,Time_EC
0,RIP,2018,8,7,08:27:00,08:30:00
1,RIP,2018,8,7,10:38:00,10:30:00
2,RIP,2018,8,7,11:57:00,12:00:00


In [13]:
Num_Request = Table_Request.shape[0]
print("Total",Num_Request,"of observations are required.")

Total 3 of observations are required.


In [14]:
[Lon,Lat,DOY,TOD,ustar,Wind_Speed,Wind_Direction,
 Air_TempC,Canopy_Height,Height_Tower,Actual_Vapor_Pressure,CO2_Density,Obukov,
 Air_Density,Specific_Heat_Capacity,Air_TempK,Incident_Solar,Net_Radiation,
 Soil_Heat_Flux,Sensible_Heat_Flux,Latent_Heat_Flux,Soil_Temp,Soil_Moisture_Content] = [
    [],[],[],[],[],[],[],
    [],[],[],[],[],
    [],[],[],[],[],
    [],[],[],[],[],[]
]

In [15]:
for irow in range(0,Num_Request):

    if Table_Request['Time_EC'][irow].minute == 30:
        print("\nWe can extract the information from the EC-tower observation table.")
        Record_tmp = Table_EC_Obs.loc[(Table_EC_Obs['Year']==Table_Request['Year'][irow])&
                              (Table_EC_Obs['Month']==Table_Request['Month'][irow])&
                              (Table_EC_Obs['Day']==Table_Request['Day'][irow])&
                              (Table_EC_Obs['Hour']==Table_Request['Time_EC'][irow].hour)&
                              ((Table_EC_Obs['Minute']==Table_Request['Time_EC'][irow].minute))]
        print("Processing date and time is:",Record_tmp['Year'].values,Record_tmp['Month'].values,
              Record_tmp['Day'].values,Record_tmp['Hour'].values,Record_tmp['Minute'].values)

        # Append most data directly
        Lon.append(Longitude)
        Lat.append(Latitude)
        DOY.append(Record_tmp.iloc[:]['DOY'].values)
        TOD.append(Record_tmp.iloc[:]['TOD'].values)
        ustar.append(Record_tmp.iloc[:]['ustar'].values)
        Wind_Speed.append(Record_tmp.iloc[:]['Wind\nSpeed'].values)
        Wind_Direction.append(Record_tmp.iloc[:]['Wind\nDirection'].values)
        Air_TempC.append(Record_tmp.iloc[:]['ASP\nTemp.'].values)
        Actual_Vapor_Pressure.append(Record_tmp.iloc[:]['Actual\nVapor\nPressure'].values)
        CO2_Density.append(Record_tmp.iloc[:]['CO2\nDensity'].values)
        Air_Density.append(Record_tmp.iloc[:]['Air\nDensity'].values)
        Specific_Heat_Capacity.append(Record_tmp.iloc[:]['Specific\nHeat'].values)
        Air_TempK.append(Record_tmp.iloc[:]['ASP\nTemp.'].values+273.15)
        Incident_Solar.append(Record_tmp.iloc[:]['Incident\nSolar'].values)
        Net_Radiation.append(Record_tmp.iloc[:]['Net\nRadiation'].values)
        Soil_Heat_Flux.append(Record_tmp.iloc[:]['Soil\nHeat Flux'].values)
        Sensible_Heat_Flux.append(Record_tmp.iloc[:]['Sensible\nHeat Flux'].values)
        Latent_Heat_Flux.append(Record_tmp.iloc[:]['Latent\nHeat Flux'].values)
        Soil_Temp.append(Record_tmp.iloc[:]['Soil\nTemp'].values)
        Soil_Moisture_Content.append(Record_tmp.iloc[:]['Soil \nMoisture\nContent'].values)
        Height_Tower.append(EC_monitoring_height)

        # Canopy height is calculated based on an assumption
        # If the DOY <=90, Canopy_Height = 0
        # If the DOY >=244, Canopy_Height = 0
        # If the 90< DOY <244, Canopy_Height = 0.0046*DOY+0.9809
        # Canopy_Height
        if Record_tmp.iloc[:]['DOY'].values <=90:
            Hcanopy = 0
        elif Record_tmp.iloc[:]['DOY'].values >=244:
            Hcanopy = 0
        else:
            Hcanopy = 0.0046*Record_tmp.iloc[:]['DOY'].values+0.9809
        Canopy_Height.append(Hcanopy)

        # Obukov length calculation
        Obukov_Length = (-Record_tmp.iloc[:]['Specific\nHeat'].values*
                         (Record_tmp.iloc[:]['ASP\nTemp.'].values+273.15)*
                         (Record_tmp.iloc[:]['Air\nDensity'].values)*
                         (Record_tmp.iloc[:]['ustar'].values**3))/(0.41*9.8*Record_tmp.iloc[:]['Sensible\nHeat Flux'].values)
        Obukov.append(Obukov_Length)
    else:
        print("\nWe need to do the interpretation between two adjacent times.")
        Record_tmp_before = Table_EC_Obs.loc[(Table_EC_Obs['Year']==Table_Request['Year'][irow])&
                                             (Table_EC_Obs['Month']==Table_Request['Month'][irow])&
                                             (Table_EC_Obs['Day']==Table_Request['Day'][irow])&
                                             (Table_EC_Obs['Hour']==Table_Request['Time_EC'][irow].hour-1)&
                                             (Table_EC_Obs['Minute']==Table_Request['Time_EC'][irow].minute+30)]
        Record_tmp_after = Table_EC_Obs.loc[(Table_EC_Obs['Year']==Table_Request['Year'][irow])&
                                            (Table_EC_Obs['Month']==Table_Request['Month'][irow])&
                                            (Table_EC_Obs['Day']==Table_Request['Day'][irow])&
                                            (Table_EC_Obs['Hour']==Table_Request['Time_EC'][irow].hour)&
                                             (Table_EC_Obs['Minute']==Table_Request['Time_EC'][irow].minute+30)]
        # The first record
        print("Processing date and time half an hour before:",Record_tmp_before['Year'].values,Record_tmp_before['Month'].values,
              Record_tmp_before['Day'].values,Record_tmp_before['Hour'].values,Record_tmp_before['Minute'].values)
        # The second record
        print("Processing date and time half an hour after:",Record_tmp_after['Year'].values,Record_tmp_after['Month'].values,
              Record_tmp_after['Day'].values,Record_tmp_after['Hour'].values,Record_tmp_after['Minute'].values)
#         print(Record_tmp_before.shape,Record_tmp_after.shape)
        # Append most data directly
        Lon.append(Longitude)
        Lat.append(Latitude)
        DOY.append(Record_tmp_before.iloc[:]['DOY'].values)
        TOD.append((Record_tmp_before.iloc[:]['TOD'].values+Record_tmp_after.iloc[:]['TOD'].values)/2)
        ustar.append((Record_tmp_before.iloc[:]['ustar'].values+Record_tmp_after.iloc[:]['ustar'].values)/2)
        Wind_Speed.append((Record_tmp_before.iloc[:]['Wind\nSpeed'].values+Record_tmp_after.iloc[:]['Wind\nSpeed'].values)/2)
        Wind_Direction.append((Record_tmp_before.iloc[:]['Wind\nDirection'].values+Record_tmp_after.iloc[:]['Wind\nDirection'].values)/2)
        Air_TempC.append((Record_tmp_before.iloc[:]['ASP\nTemp.'].values+Record_tmp_after.iloc[:]['ASP\nTemp.'].values)/2)
        Actual_Vapor_Pressure.append((Record_tmp_before.iloc[:]['Actual\nVapor\nPressure'].values+Record_tmp_after.iloc[:]['Actual\nVapor\nPressure'].values)/2)
        CO2_Density.append((Record_tmp_before.iloc[:]['CO2\nDensity'].values+Record_tmp_after.iloc[:]['CO2\nDensity'].values)/2)
        Air_Density.append((Record_tmp_before.iloc[:]['Air\nDensity'].values+Record_tmp_after.iloc[:]['Air\nDensity'].values)/2)
        Specific_Heat_Capacity.append((Record_tmp_before.iloc[:]['Specific\nHeat'].values+Record_tmp_after.iloc[:]['Specific\nHeat'].values)/2)
        Air_TempK.append((Record_tmp_before.iloc[:]['ASP\nTemp.'].values+273.15+Record_tmp_after.iloc[:]['ASP\nTemp.'].values+273.15)/2)
        Incident_Solar.append((Record_tmp_before.iloc[:]['Incident\nSolar'].values+Record_tmp_after.iloc[:]['Incident\nSolar'].values)/2)
        Net_Radiation.append((Record_tmp_before.iloc[:]['Net\nRadiation'].values+Record_tmp_after.iloc[:]['Net\nRadiation'].values)/2)
        Soil_Heat_Flux.append((Record_tmp_before.iloc[:]['Soil\nHeat Flux'].values+Record_tmp_after.iloc[:]['Soil\nHeat Flux'].values)/2)
        Sensible_Heat_Flux.append((Record_tmp_before.iloc[:]['Sensible\nHeat Flux'].values+Record_tmp_after.iloc[:]['Sensible\nHeat Flux'].values)/2)
        Latent_Heat_Flux.append((Record_tmp_before.iloc[:]['Latent\nHeat Flux'].values+Record_tmp_after.iloc[:]['Latent\nHeat Flux'].values)/2)
        Soil_Temp.append((Record_tmp_before.iloc[:]['Soil\nTemp'].values+Record_tmp_after.iloc[:]['Soil\nTemp'].values)/2)
        Soil_Moisture_Content.append((Record_tmp_before.iloc[:]['Soil \nMoisture\nContent'].values+Record_tmp_after.iloc[:]['Soil \nMoisture\nContent'].values)/2)
        Height_Tower.append(EC_monitoring_height)

        # Canopy height is calculated based on an assumption
        # If the DOY <=90, Canopy_Height = 0
        # If the DOY >=244, Canopy_Height = 0
        # If the 90< DOY <244, Canopy_Height = 0.0046*DOY+0.9809
        # Canopy_Height
        if Record_tmp_before.iloc[:]['DOY'].values <=90:
            Hcanopy = 0
        elif Record_tmp_before.iloc[:]['DOY'].values >=244:
            Hcanopy = 0
        else:
            Hcanopy = 0.0046*Record_tmp_before.iloc[:]['DOY'].values+0.9809
        Canopy_Height.append(Hcanopy)

        # Obukov length calculation
        Obukov_Length = (-0.5*(Record_tmp_before.iloc[:]['Specific\nHeat'].values+Record_tmp_after.iloc[:]['Specific\nHeat'].values)*
                         0.5*((Record_tmp_before.iloc[:]['ASP\nTemp.'].values+273.15)+(Record_tmp_after.iloc[:]['ASP\nTemp.'].values+273.15))*
                         0.5*(Record_tmp_before.iloc[:]['Air\nDensity'].values+Record_tmp_after.iloc[:]['Air\nDensity'].values)*
                         ((0.5*(Record_tmp_before.iloc[:]['ustar'].values+Record_tmp_after.iloc[:]['ustar'].values))**3))/(0.41*9.8*
                                                                                                                           0.5*(Record_tmp_before.iloc[:]['Sensible\nHeat Flux'].values+Record_tmp_after.iloc[:]['Sensible\nHeat Flux'].values))
        Obukov.append(Obukov_Length)


We can extract the information from the EC-tower observation table.
Processing date and time is: [2018.] [8.] [7.] [8.] [30.]

We can extract the information from the EC-tower observation table.
Processing date and time is: [2018.] [8.] [7.] [10.] [30.]

We need to do the interpretation between two adjacent times.
Processing date and time half an hour before: [2018.] [8.] [7.] [11.] [30.]
Processing date and time half an hour after: [2018.] [8.] [7.] [12.] [30.]


In [19]:
# Prepare the dataframe and save it as a CSV file
df_out = pd.DataFrame()
df_out['Site'] = Table_Request['Site']
df_out['Longitude'] = Lon
df_out['Latitude'] = Lat
df_out['Year'] = Table_Request['Year']
df_out['Month'] = Table_Request['Month']
df_out['Day'] = Table_Request['Day']
df_out['Time_flight'] = Table_Request['Time_flight']
df_out['Time_EC'] = Table_Request['Time_EC']

df_out['DOY'] = DOY
df_out['TOD'] = TOD
df_out['ustar'] = ustar
df_out['Wind_Speed'] = Wind_Speed
df_out['Wind_Direction'] = Wind_Direction
df_out['Air_TempC'] = Air_TempC
df_out['Actual_Vapor_Pressure'] = Actual_Vapor_Pressure
df_out['CO2_Density'] = CO2_Density
df_out['Air_Density'] = Air_Density
df_out['Specific_Heat_Capacity'] = Specific_Heat_Capacity
df_out['Air_TempK'] = Air_TempK
df_out['Incident_Solar'] = Incident_Solar
df_out['Net_Radiation'] = Net_Radiation
df_out['Soil_Heat_Flux'] = Soil_Heat_Flux
df_out['Sensible_Heat_Flux'] = Sensible_Heat_Flux
df_out['Latent_Heat_Flux'] = Latent_Heat_Flux
df_out['Soil_Temp'] = Soil_Temp
df_out['Soil_Moisture_Content'] = Soil_Moisture_Content
df_out['Canopy_Height'] = Canopy_Height
df_out['Height_Tower'] = Height_Tower
df_out['Obukov'] = Obukov

for col in df_out:
    df_out[col] =df_out[col].astype(str).str.replace("[","").str.replace("]","")

df_out.to_csv(dir_output + "\\" + NameCSV + '.csv', index=False)

<ipython-input-19-9bfaf2368521>:31: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_out[col] =df_out[col].astype(str).str.replace("[","").str.replace("]","")
